# Unit Square Geometry File

# Demo: Solving Laplace’s Equation on a Unit Square Mesh

In [1]:
"""This demo program solves Laplace's equation

    - div grad u(x, y) = 0

on a unit square mesh with four Dirichlet boundary conditions.
"""
# Begin demo

from dolfin import *

# Create mesh and define function space
mesh = Mesh("square.xml")
subdomains = MeshFunction("size_t", mesh, "square_facet_region.xml")
V = FunctionSpace(mesh, "Lagrange", 1)

# Define boundary condition
u1 = Expression("x[1]")
u2 = Expression("x[0] + 1")
u3 = Expression("x[1] + 1")
u4 = Expression("x[0]")

bc1 = DirichletBC(V, u1, subdomains, 1)
bc2 = DirichletBC(V, u2, subdomains, 2)
bc3 = DirichletBC(V, u3, subdomains, 3)
bc4 = DirichletBC(V, u4, subdomains, 4)

bcs = [bc1, bc2, bc3, bc4]

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Expression("0.0")
g = Expression("sin(10*x[0])")
a = inner(grad(u), grad(v))*dx
L = f*v*dx #+ g*v*ds

# Compute solution
u = Function(V)
solve(a == L, u, bcs)

# Save solution in VTK format
file = File("laplace_square.pvd")
file << u

# Plot solution
plot(u, interactive=True)



ImportError: No module named dolfin

# Cape Geometry File

<img src="cape.jpg">

# Demo: Solving Laplace's Equation on a Cape Mesh

In [ ]:
"""This demo program solves Laplace's equation

    - div grad u(x, y) = 0

on the cape mesh with four Dirichlet boundary conditions.
"""
# Begin demo

from dolfin import *

# Create mesh and define function space
mesh = Mesh("cape.xml")
subdomains = MeshFunction("size_t", mesh, "cape_facet_region.xml")
V = FunctionSpace(mesh, "Lagrange", 1)

# Define boundary condition
u1 = Expression("x[1] + 1")
u2 = Expression("x[0]")
u3 = Expression("x[1]")
u4 = Expression("x[0] + x[1]")

bc1 = DirichletBC(V, u1, subdomains, 1)
bc2 = DirichletBC(V, u2, subdomains, 2)
bc3 = DirichletBC(V, u3, subdomains, 3)
bc4 = DirichletBC(V, u4, subdomains, 4)

bcs = [bc1, bc2, bc3, bc4]

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
#f = Expression("10*exp(-(pow(x[0] - 1.0, 2) + pow(x[1] - 0.5, 2)) / 0.02)")
f = Expression("0.0")
g = Expression("sin(10*x[0])")
a = inner(grad(u), grad(v))*dx
L = f*v*dx #+ g*v*ds

# Compute solution
u = Function(V)
solve(a == L, u, bcs)

# Save solution in VTK format
file = File("laplace_cape.pvd")
file << u

# Plot solution
plot(u, interactive=True)



# Demo: Enforcing Periodic Boundary Conditions

In [ ]:
"""This demo program solves Poisson's equation

    - div grad u(x, y) = f(x, y)

on the unit square with source f given by

    f(x, y) = exp(-((x - 0.5)^2 + (y - 0.5)^2) / 0.02)

and boundary conditions given by

    u(x, y) = 0        for x = 0 or x = 1
du/dn(x, y) = sin(5*x) for y = 0 or y = 1
"""

# Begin demo

from dolfin import *

class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT on one of the two corners (0, 1) and (1, 0)
        return bool((near(x[0], 0) or near(x[1], 0)) and 
                (not ((near(x[0], 0) and near(x[1], 1)) 
                        or 
                        (near(x[0], 1) and near(x[1], 0)))
                        ) and on_boundary)

    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:   # near(x[1], 1)
            y[0] = x[0]
            y[1] = x[1] - 1.
            

# Create mesh and define function space
mesh = Mesh("square.xml")
subdomains = MeshFunction("size_t", mesh, "square_facet_region.xml")
pbc = PeriodicBoundary()
V = FunctionSpace(mesh, "Lagrange", 1, constrained_domain=pbc)

# boundary conditions
u0 = Constant(0.0)
bc0 = DirichletBC(V, u0, subdomains, 2)
bc1 = DirichletBC(V, u0, subdomains, 4)
bcs = [bc0, bc1]

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Expression("exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2))/ 0.02)")
g = Expression("sin(5*x[0])")
a = inner(grad(u), grad(v))*dx
L = f*v*dx

# Compute solution
u = Function(V)
solve(a == L, u, bcs)

# Save solution in VTK format
file = File("poisson_square.pvd")
file << u

# Plot solution
plot(u, interactive=True)
